In [ ]:
import numpy as np
import torch

from src.core.anchor import unmap, expand_binary_labels
from src.core.bbox import PseudoSampler
from src.core.utils import describe

from src.quiz.quiz6 import anchor_inside_flags
from src.quiz.quiz7 import bbox2delta
from src.quiz.quiz11 import assign

%store -r anchor_target_variable

In [ ]:
(anchor_list, valid_flag_list, gt_bboxes, gt_labels, img_metas, train_cfg, label_channels) = anchor_target_variable

In [ ]:
print('anchor_list : ', describe(anchor_list))
print('valid_flag_list : ', describe(valid_flag_list))
print('gt_bboxes : ', describe(gt_bboxes))
print('gt_labels : ', describe(gt_labels))
print('img_metas : ', describe(img_metas))
print('train_cfg : ', describe(train_cfg))

In [ ]:
"""
배치 내의 여러 장의 이미지중 한 장의 이미지에 대해서 동작하는 모습을 보여주는 예제입니다. 
현재 배치 크기는 1 입니다.
"""

flat_anchors = anchor_list[0]
valid_flags = valid_flag_list[0]
gt_bboxes = gt_bboxes[0]
gt_labels = gt_labels[0]
img_meta = img_metas[0]
cfg = train_cfg

inside_flags = anchor_inside_flags(flat_anchors, valid_flags,
                                   img_meta['img_shape'][:2],
                                   cfg.allowed_border)
anchors = flat_anchors[inside_flags, :]

In [ ]:
assign_result = assign(anchors, gt_bboxes, cfg.assigner)

"""
RetinaNet은 Positive sample을 전량 사용하므로 Sampling을 진행하지 않습니다.
MMDetection은 Sampling을 사용하는 타 모델과 RetinaNet을 함께 구현하기 위하여 PseudoSampler 클래스를 사용합니다.
해당 클래스를 통해 Sampling을 진행한 것과 같은 형식으로 만들 수 있습니다.
"""
bbox_sampler = PseudoSampler()
sampling_result = bbox_sampler.sample(assign_result, anchors,
                                      gt_bboxes)

In [ ]:
"""
Sampling한 결과물을 학습에 사용할 target형태로 만들어주는 작업입니다.

첫 번째 단계에서는 모든 anchor에서 target을 계산할 수 있다는 가정 하에
bbox_targets를 anchors와 같게,
labels를 anchor의 개수와 같게 만들고
값을 모두 0으로 초기화 합니다.(torch.zeros)
"""
num_valid_anchors = anchors.shape[0]
bbox_targets = torch.zeros_like(anchors)
bbox_weights = torch.zeros_like(anchors)
labels = anchors.new_zeros(num_valid_anchors, dtype=torch.long)
label_weights = anchors.new_zeros(num_valid_anchors, dtype=torch.float)

In [ ]:
"""
두 번째 단계는 target에 적절한 값을 대입해주는 작업입니다.
학습에 사용할 anchor의 index를 pos_inds, neg_inds에서 각각 가져옵니다.
해당 anchor들을 delta 형태로 고친 다음, targets에 넣습니다.
"""
pos_inds = sampling_result.pos_inds
neg_inds = sampling_result.neg_inds

if len(pos_inds) > 0:
    pos_bbox_targets = bbox2delta(sampling_result.pos_bboxes,
                                  sampling_result.pos_gt_bboxes)
    bbox_targets[pos_inds, :] = pos_bbox_targets
    bbox_weights[pos_inds, :] = 1.0

    labels[pos_inds] = gt_labels[sampling_result.pos_assigned_gt_inds]
    label_weights[pos_inds] = 1.0
    
if len(neg_inds) > 0:
    label_weights[neg_inds] = 1.0

"""
처음 만들었던 flat_anchors와 같은 모양으로 복구하기 위하여
inside_flag로 제외했던 anchor에 대한 target을 추가합니다.
학습에 사용되지 않으므로 모두 0으로 채우게 됩니다.
"""

num_total_anchors = flat_anchors.size(0)
labels = unmap(labels, num_total_anchors, inside_flags)
label_weights = unmap(label_weights, num_total_anchors, inside_flags)
if label_channels > 1:
    labels, label_weights = expand_binary_labels(
            labels, label_weights, label_channels)
bbox_targets = unmap(bbox_targets, num_total_anchors, inside_flags)
bbox_weights = unmap(bbox_weights, num_total_anchors, inside_flags)

describe((labels, label_weights, bbox_targets, bbox_weights, pos_inds,
        neg_inds))